Primer red neuronal

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Importar los datasets
ratings = pd.read_csv("./archive/Ratings.csv")
books = pd.read_csv("./archive/Books.csv")

# Convertir los ratings a formato numérico
ratings["Book-Rating"] = ratings["Book-Rating"].astype("float32")

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
(x_train, y_train), (x_test, y_test) = train_test_split(ratings, test_size=0.25)

# Convertir los datos a formato numérico
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

# Embedding layers for users and books
user_embedding = tf.keras.layers.Embedding(input_dim=x_train.shape[0], output_dim=64)
book_embedding = tf.keras.layers.Embedding(input_dim=x_train.shape[1], output_dim=64)

# Dense layer for predicting ratings
dense = tf.keras.layers.Dense(1, activation='sigmoid')

# Compile the model
model = tf.keras.Model(inputs=[user_embedding, book_embedding], outputs=dense)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
model.fit(x_train, y_train, epochs=10)

# Evaluate the model
model.evaluate(x_test, y_test)

# Make predictions
predictions = model.predict(x_test)

# Imprimir las predicciones
print(predictions)

C:\Users\sebas\AppData\Local\Temp\ipykernel_23672\2134458638.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("./archive/Books.csv")


ValueError: not enough values to unpack (expected 4, got 2)